<a href="https://colab.research.google.com/github/albertwujj/gptransfer/blob/master/gptransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# I put the final embedding output through a fully-connected layer and train on binary cross-entropy,
# and freeze the earlier layers of the decoder and add regularization.

# I haven't finished experimenting with this particular fine-tuning method yet, will add comments

from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=True)

PROJ = "My Drive/ColabExperiments/"
PROJECT_PATH = join(ROOT, PROJ)
%cd "{PROJECT_PATH}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/ColabExperiments


In [0]:
from importlib.machinery import SourceFileLoader
model = SourceFileLoader('model', 'gpt-2/src/model.py').load_module()
encoder = SourceFileLoader('encoder', 'gpt-2/src/encoder.py').load_module()
sample = SourceFileLoader('sample', 'gpt-2/src/sample.py').load_module()

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import sys
np.set_printoptions(precision=3,threshold=sys.maxsize)
import gc

from itertools import islice
import json
import os
import random
from timeit import default_timer as timer

seed = None
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [0]:
# Prepare data cell

batch_size = 20

from sklearn.model_selection import train_test_split
from encoder import Encoder

def read_amazon_kaggle(filename, lines=None):
    with open(filename) as f:
        y_x = [(1 if line[9] == '2' else 0, line[10:].strip()) for line in f]
        if lines:
            y_x = random.sample(y_x, lines)
        y, x = zip(*y_x)
        return x, y
    
def read_toxicity(filename, lines=None):
    with open(filename, newline='') as f:
        reader = csv.reader(f, delimiter=',', quotechar='\"')
        next(reader)
        y_x = [(float(row[1]), row[2].strip('\"')) for row in reader]
        if lines:
            y_x = random.sample(y_x, lines)
        y, x = zip(*y_x)
        return x, y
    
def get_encoder(model_name):
    with open(os.path.join('gpt-2/models', model_name, 'encoder.json'), 'r') as f:
        encoder = json.load(f)
    with open(os.path.join('gpt-2/models', model_name, 'vocab.bpe'), 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    return Encoder(
        encoder=encoder,
        bpe_merges=bpe_merges,
    )

def encode(x, y, model_name='117M'):
    enc = get_encoder(model_name)
    xcode = []
    for i, x in enumerate(x):
        vec = enc.encode(x)
        xcode.append(vec)
    x = xcode
    return zip(*sorted(zip(x, y), key = lambda t: len(t[0])))
      
def get_data(x_raw, y_raw):
  
    x_all, y_all = encode(x_raw, y_raw)
    x_all, y_all = (np.asarray(x_all), np.asarray(y_all))
    return x_all, y_all

def mod_by_batchsize(x):
    l = (len(x) // batch_size) * batch_size
    x = x[:l]
    return x
    


lines = 30000
#x_raw, y_raw = read_toxicity('gptransfer/data/jigsaw_toxicity/train.csv')
x_raw, y_raw = read_amazon_kaggle('gptransfer/data/amazonreviews/train.ft.txt', lines)
x_all, y_all = get_data(x_raw, y_raw)



In [0]:
# Hyper Param definition cell

frozen_layers = 8 # how many GPT2 layers to not train (out of 12)
all_dropout = False
lr = 0.0001



In [0]:
# This cell contains all code copied from GPT2. Original GPT2 model with slight modifications.

from model import shape_list, positions_for, norm, default_hparams, block, gelu, conv1d

dropout_prob = None
curr_layer = None

    
def mlp_dropout(x, scope, n_state, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        h = gelu(conv1d(x, 'c_fc', n_state))
        if all_dropout and curr_layer > frozen_layers:
            h = tf.nn.dropout(h, rate=dropout_prob)
        h2 = conv1d(h, 'c_proj', nx)
        if curr_layer > frozen_layers:
            h2 = tf.nn.dropout(h2, rate=dropout_prob)
        return h2
    
    
model.mlp = mlp_dropout

def custom_model(hparams, X, past=None, scope='model', reuse=False):
    global dropout_prob, curr_layer
    dropout_prob = tf.placeholder_with_default(0.0, shape=())
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wpe = tf.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.01))
        wte = tf.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.02))
        past_length = 0 if past is None else tf.shape(past)[-2]
        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))
        
        # Transformer
        presents = []
        pasts = tf.unstack(past, axis=1) if past is not None else [None] * hparams.n_layer
        assert len(pasts) == hparams.n_layer
        for layer, past in enumerate(islice(pasts, hparams.n_layer)):
            curr_layer=layer
            h, present = block(h, 'h%d' % layer, past=past, hparams=hparams)
            if layer == frozen_layers:
                h = tf.stop_gradient(h)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits

        results['h'] = h
        return results
 
def small_model(hparams, X, **kwargs):
    hparams.n_layer = 5
    return custom_model(hparams, X, **kwargs)
    

In [0]:
from keras.preprocessing.sequence import pad_sequences

def load_model(model_name='117M'):
    with open(os.path.join('gpt-2/models', model_name, 'hparams.json')) as f:
        hparams = model.default_hparams()
        hparams.override_from_dict(json.load(f))   
        def step(hparams, tokens, past=None):
            lm_output = custom_model(hparams=hparams, X=tokens, past=None, reuse=tf.AUTO_REUSE)
            h = lm_output['h']
            return {
                'h': h
            }
        
        X = tf.placeholder(tf.int32, [batch_size, None])
        outputs = step(hparams, X) # (batch, sequence, embedding)
        
        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('gpt-2/models', model_name))
        saver.restore(sess, ckpt)
        
    return X, outputs

def add_binary_finetune(outputs):
    with tf.variable_scope('binary_finetune'):
        
        final_embd = outputs['h'][:,-1,:]
        w = tf.get_variable('w', (final_embd.shape[-1],), initializer=tf.contrib.layers.xavier_initializer(uniform=False))
        b = tf.get_variable('b', (final_embd.shape[0],), initializer=tf.constant_initializer(0))
        sess.run(tf.variables_initializer([w,b]))
        l2loss = tf.nn.l2_loss(w)
        logits = tf.tensordot(final_embd,w, [[1],[0]], name='z') + b
        ypred = tf.nn.sigmoid(logits)
        
        ytrue = tf.placeholder(tf.float32, (batch_size,))
      
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=ytrue, logits=logits) + l2loss * .02)
        
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = lr
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           800//batch_size, 0.95, staircase=False)
        optim = tf.train.AdamOptimizer(learning_rate=lr)
        minimize = optim.minimize(loss)
        sess.run(tf.variables_initializer(optim.variables() + [global_step]))
        
    return ytrue, ypred, minimize, optim


def get_train_infer(X, outputs, ytrue, ypred, minimize):
    
    def train(dataX, labels):
        for i in range(0, len(dataX), batch_size):
            xfeed = pad_sequences(dataX[i:i+batch_size])
            yfeed = labels[i:i+batch_size]
            sess.run(minimize, feed_dict={X:xfeed, ytrue:yfeed, dropout_prob: 0.5})

    def infer(dataX):
        preds = []
        for i in range(0, len(dataX), batch_size):
            xfeed = pad_sequences(dataX[i:i+batch_size])
            predbatch = sess.run(ypred, feed_dict={X: xfeed})
            preds.append(predbatch)
                
        preds = np.concatenate(preds, axis=0)
        return preds

    return train, infer


In [0]:
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.contrib.memory_stats.python.ops.memory_stats_ops import MaxBytesInUse
from sklearn.model_selection import KFold as KFold
from itertools import product

import multiprocess as mp # must spawn new process after clearing GPU mem


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = None

import pickle
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from numba import cuda
from functools import reduce
import sklearn

def loadTrials(): 
    trials = pickle.load(open("gptransfer/results/trials.p", "rb"))
    # delete all 'running' jobs- jobs stopped in execution
    trials._dynamic_trials = [t for t in trials._dynamic_trials if t['state'] != 1 and t['state'] != 3]
    for i, t in enumerate(trials._dynamic_trials):
        t['tid'] = i
    trials._ids = set()
    trials.refresh()
    return trials

def main():

    space = {'all_dropout': False, 'lr': hp.loguniform('learning_rate', np.log(.00001), np.log(.001)),'frozen_offset':hp.randint('fo',3)}
    trials = loadTrials()
    
    print(len(trials.trials))
    with open('gptransfer/results/out', 'w', buffering=1) as out:
        printout = lambda w: out.write(w + '\n')

        def objective(space):
            print(f'trial number {len(trials.trials)}')
            pickle.dump(trials, open('gptransfer/results/trials.p', 'wb'))
            global all_dropout, lr, frozen_layers
            all_dropout = space['all_dropout']
            lr = space['lr']
            frozen_offset = space['frozen_offset']
            frozen_layers = 5 + frozen_offset
            configs_string = reduce(lambda x,y:x+y,[f'{k} {v} ' for k, v in space.items()])
            print(configs_string)
            
            kf = KFold(n_splits=3,random_state=None, shuffle=True)
            accuracies = []
            for train_index, val_index in islice(kf.split(x_all, y_all), 2):
                x_train, y_train, x_val, y_val = [mod_by_batchsize(arr[inds]) for inds, arr in product([train_index, val_index], [x_all, y_all])]
                
                def run_fold(acc):
                    with tf.device('/device:GPU:0') as dev, tf.Session(graph=tf.get_default_graph(),
                                                                       config=tf.ConfigProto(allow_soft_placement=True)) as sessl:
                        global sess
                        sess = sessl
                        max_bytes_in_use = MaxBytesInUse()

                        X, outputs = load_model()
                        ytrue, ypred, minimize, optim = add_binary_finetune(outputs)
                        train, infer = get_train_infer(X, outputs, ytrue, ypred, minimize)

                        eval_batchsize = 100000000000
                        eval_batchsize = (eval_batchsize//batch_size) * batch_size
                        start = timer()

                        for i in range(0, x_train.shape[0], eval_batchsize):

                            end = min(x_train.shape[0], i+eval_batchsize)
                            x_train_b = x_train[i:end]
                            y_train_b = y_train[i:end]
                            train(x_train_b, y_train_b)

                            pred_val = infer(x_val)
                            corrects = np.count_nonzero(np.sign(y_val - .5) == 
                                                        np.sign(pred_val - .5)) / y_val.shape[0]

                            gb_used = sess.run(max_bytes_in_use) / 1e9
                            
                            printout(f'GPU GB used {i//eval_batchsize}: {gb_used:.2f}')
                            print(f'GPU GB used {i//eval_batchsize}: {gb_used:.2f}')
                            print(f'elapsed minutes {i//eval_batchsize}: {(timer() - start)//60}')

                        accuracy = np.count_nonzero(np.sign(y_val - .5) == np.sign(pred_val - .5)) / y_val.shape[0] if type(y_val.tolist()[0]) == int else sklearn.metrics.log_loss(y_val, pred_val)
                        print('Fold Complete')
                        print(f'Accuracy: {accuracy}')
                        acc.value = accuracy
                        cuda.select_device(0)
                        cuda.close()
                
                acc = mp.Value('f',0.0)
                p = mp.Process(target=run_fold, args=(acc,))
                
                p.start()
                p.join()
                if 0 and acc.value < .3: # too bad, don't do next
                    return {'loss': -acc.value, 'status': STATUS_OK}
                accuracies.append(acc.value)
                
            %cp gptransfer/results/out gptransfer/results/out_backup
            avg_accuracy = sum(accuracies) / len(accuracies) + frozen_offset * .002
            print(f'Average accuracy with {configs_string}: {avg_accuracy}\n\n')
            printout(f'Average accuracy with {configs_string}: {avg_accuracy}\n\n')
            return {'loss': -avg_accuracy, 'status': STATUS_OK}

        best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=2000, show_progressbar=False)
        pickle.dump(trials, open('gptransfer/results/trials.p', 'wb'))

        print(accuracies)
        
if __name__=='__main__':
    main()
    
# all_dropout False frozen_offset 0 lr 0.0001266190503028293 
   

[{'loss': -0.9430999972820282, 'status': 'ok'}, {'loss': -0.9459499933719635, 'status': 'ok'}, {'status': 'new'}]
2
trial number 3
all_dropout False frozen_offset 1 lr 3.0476704402180002e-05 
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gpt-2/models/117M/model.ckpt
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
GPU GB used 0: 6.29
elapsed minutes 0: 5.0
Fold Complete
Accuracy: 0.9353
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gpt-2/models/117M/model.ckpt
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
GPU GB used 0: 5.97
el

In [0]:
import pickle
from hyperopt import Trials
trials = pickle.load(open("gptransfer/results/trials.p", "rb"))
print(trials.trials[0]['result'])